<a href="https://colab.research.google.com/github/GiX007/imdb-sql-lab/blob/main/NLIDBs_Lab_DB_Systems_2025_DSIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The goal of this exercise is to get familiar with the essentials of NLIDBs with LLMs.
This notebook is structured to guide us through the 3 main problems of Text-to-SQL, SQL-to-Text, and Data-to-Text.
At the end of this exercise, we will have a better grasp on how LLMs can be used to perform these complex tasks and the basics on how to use them.

**<u>To start the excercise</u>: we need an OpenAI API key. We must enter this API key in the code cell bellow. This key is not shareable with anyone else.**

**Technicalities**:

- This notebook is created to work with both the OpenAI API and the vLLM library.
- During this lab, we will use an OpenAI API through an API key to execute some calls to the LLM. Have in mind that each call is charged.
- vLLM is a library that handles loading the LLM on the GPU and using it to generate answers from the prompts that we provide.
- No matter which backend we choose, we can use the LLM through the `llm_generate` function which takes care of using the right service.

In [ ]:
LLM_BACKEND = "openai"
OPENAI_API_KEY = ""
# LLM_BACKEND = "vllm"

## Install Dependencies

In [ ]:
if LLM_BACKEND == "vllm":
  !pip install vllm bitsandbytes>=0.44.0 tabulate
elif LLM_BACKEND == "openai":
  !pip install openai
else:
  raise ValueError(f"LLM backend '{LLM_BACKEND}' not supported!")

## Prepare LLM

In [ ]:
if LLM_BACKEND == "vllm":
  from vllm import LLM, SamplingParams
  import torch

  # If we use the official llama checkpoint the students must ask for access and login
  # !huggingface-cli login
  # model_id = "meta-llama/Llama-3.1-8B-Instruct"

  model_id = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
  llm = LLM(model=model_id, dtype=torch.float16, trust_remote_code=True, quantization="bitsandbytes", load_format="bitsandbytes", max_model_len=8000)

  def vllm_generate():
    sampling_params = SamplingParams(max_tokens=512)
    # Pass the chat to the LLM so it can generate its repsonse
    outputs = llm.chat(conversation, sampling_params=sampling_params)
    # Get the generated text from the response
    generated_text = outputs[0].outputs[0].text

    return generated_text

  llm_generate = vllm_generate

elif LLM_BACKEND == "openai":
  from openai import OpenAI
  from typing import Dict, List

  def gpt_generate(chat: List[Dict[str, str]]):
    client = OpenAI(api_key=OPENAI_API_KEY)

    completion = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=chat
    )

    generated_text = completion.choices[0].message.content

    return generated_text

  llm_generate = gpt_generate
else:
  raise ValueError(f"LLM backend '{LLM_BACKEND}' not supported!")

## Aditional imports and helper functions

In [ ]:
from IPython.display import display, Markdown

## Download the Spider dataset

In [ ]:
!gdown 1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
!unzip spider_data.zip

import pandas as pd
import json

# Load dev set
spider_dev = pd.read_json("spider_data/dev.json")
spider_dev = spider_dev[["question", "query", "db_id"]]
# Load DB schemas
with open("spider_data/tables.json", "r") as fp:
  schema_list = json.load(fp)
schema_dict = {schema["db_id"]: schema for schema in schema_list}

Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=6bf216eb-0480-4ac2-9511-6bd212a882fc
To: /content/spider_data.zip
100% 206M/206M [00:03<00:00, 68.3MB/s]
Archive:  spider_data.zip
   creating: spider_data/
  inflating: spider_data/dev_gold.sql  
  inflating: __MACOSX/spider_data/._dev_gold.sql  
   creating: spider_data/database/
  inflating: __MACOSX/spider_data/._database  
  inflating: spider_data/.DS_Store   
  inflating: __MACOSX/spider_data/._.DS_Store  
  inflating: spider_data/test_tables.json  
  inflating: __MACOSX/spider_data/._test_tables.json  
  inflating: spider_data/train_others.json  
  inflating: __MACOSX/spider_data/._train_others.json  
  inflating: spider_data/train_spider.json  
  inflating: __MACOSX/spider_data/._train_spider.json  
  inflating: spider_data/test.json   
  inflating: __MACOSX/spider_data/._test.json  
 

## Try usign the LLM

Before delving into NLIDBs, let us first take a look on how we use an LLM.
When we prompt the LLM, we give it a the start of a conversation and ask it to generate a response.
In the example below, we have two parts in the conversation:

- A **"system"** message that we call `instructions`: System messages are introductory information that describe the role of the LLM and how it should behave. In this example, we specify that the LLM is a pirate and it should respond in such a manner.

- A **"user"** message that we call `question`: User messages are parts of the conversation that were expressed by the user (i.e., us). These messages are usually asking the LLM to perform a task or present a question that should be answered.

There is an additional message role called **"assistant"**: This role indicates that this part of the conversation is a response that was generated by the assistant (i.e., the LLM).
This is useful when we have a long conversation with multiple question-answer turns, or when we want to perform few-shot prompting.
We will se an example usage of this role later in the exercise with few-shot prompting.

We will play around with the `instructions` and `question` variables and see what answers we get from the LLM.



In [ ]:
instructions =\
"""
You are a very kind pirate that responds to all questions helpfully but only in pirate lingo.
"""

question =\
"""
Can you tell me a few things about Text-to-SQL?
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)

display(generated_text)

'Arrr, matey! Text-to-SQL be a mighty fine tool in the digital seas. It be a way to turn plain ol\' English talk into commands that a database can understand—like makin\' the words of a landlubber into the language of the seven seas. \n\nNow, with Text-to-SQL, ye can ask questions like "How many treasure chests be buried in the island?" and it’ll translate yer fairest words into a proper SQL query, so ye can find the loot ye seek without fuss or bother.\n\nIt be a boon fer sailors and landlubbers alike who wish to chart the treacherous waters of data without learnin\' the arcane dialect of SQL. \n\nAny other treasures of knowledge I can help ye find, matey?'

# Part 1: Text-to-SQL

## 1.1: Getting familiar with the data

The first task we will explore is Text-to-SQL, which can enable NL querying over a database.
We have already loaded the Spider dataset - a famous Text-to-SQL dataset - to try.

In the code cells bellow, we can get familiar with the dataset and its structure.
Take a look at the two loaded variables:
- `spider_dev`: A dataframe that contains all the NL/SQL pairs of the dev split of the dataset and the id of the database that it corresponds to.
- `schema_dict`: A dictionary mapping database ids to the corresponding database info (table names, column names, etc.)

We explore the Spider dataset and get familiar with its structure.
We pick one example in the `spider_dev` dataframe and have a look at its DB schema by looking up the corresponding `db_id` in the `schema_dict` dictionary.
In the next step, we will use this example to explore how an LLM can perform Text-to-SQL.

In [ ]:
spider_dev

,question,query,db_id
0,How many singers do we have?,SELECT count(*) FROM singer,concert_singer
1,What is the total number of singers?,SELECT count(*) FROM singer,concert_singer
2,"Show name, country, age for all singers ordere...","SELECT name , country , age FROM singer ORDE...",concert_singer
3,"What are the names, countries, and ages for ev...","SELECT name , country , age FROM singer ORDE...",concert_singer
4,"What is the average, minimum, and maximum age ...","SELECT avg(age) , min(age) , max(age) FROM s...",concert_singer
...,...,...,...
1029,What are the citizenships that are shared by s...,SELECT Citizenship FROM singer WHERE Birth_Yea...,singer
1030,How many available features are there in total?,SELECT count(*) FROM Other_Available_Features,real_estate_properties
1031,What is the feature type name of feature AirCon?,SELECT T2.feature_type_name FROM Other_Availab...,real_estate_properties
1032,Show the property type descriptions of propert...,SELECT T2.property_type_description FROM Prope...,real_estate_properties


In [ ]:
schema_dict["flight_2"]

{'column_names': [[-1, '*'],
  [0, 'airline id'],
  [0, 'airline name'],
  [0, 'abbreviation'],
  [0, 'country'],
  [1, 'city'],
  [1, 'airport code'],
  [1, 'airport name'],
  [1, 'country'],
  [1, 'country abbrev'],
  [2, 'airline'],
  [2, 'flight number'],
  [2, 'source airport'],
  [2, 'destination airport']],
 'column_names_original': [[-1, '*'],
  [0, 'uid'],
  [0, 'Airline'],
  [0, 'Abbreviation'],
  [0, 'Country'],
  [1, 'City'],
  [1, 'AirportCode'],
  [1, 'AirportName'],
  [1, 'Country'],
  [1, 'CountryAbbrev'],
  [2, 'Airline'],
  [2, 'FlightNo'],
  [2, 'SourceAirport'],
  [2, 'DestAirport']],
 'column_types': ['text',
  'number',
  'text',
  'text',
  'text',
  'text',
  'text',
  'text',
  'text',
  'text',
  'number',
  'number',
  'text',
  'text'],
 'db_id': 'flight_2',
 'foreign_keys': [[13, 6], [12, 6]],
 'primary_keys': [1, 6, 10],
 'table_names': ['airlines', 'airports', 'flights'],
 'table_names_original': ['airlines', 'airports', 'flights']}

## 1.2: Creating a first LLM prompt for Text-to-SQL
Now that we are more familiar with the data, let's try to use a LLM for Text-to-SQL.
In the code block bellow, we will complete the intructions and question parts of the prompt with all the necessary info for the LLM to generate the SQL.
Similarly to the introductory example, the `instructions` are more general guidelines about how the LLM should behave (e.g., "You are an expert in SQL", "You only respond with correct SQL queries", etc.), while the `question` contains all the information that is relevant to the specific example of the task it must solve.


We create a prompt for Text-to-SQL with the example we chose in the previous step. We also need to try to think of the information that is necessary to tranlsate a given NLQ to SQL. We must include this information to the prompt in a clear way so that the model can correctly generate the SQL.

In [ ]:
instructions =\
"""
You are an expert in SQL and you always respond with correct SQL queries!
"""

question =\
"""
Can you tell me how many Airlines from France exist?
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)
print(generated_text)

```sql
SELECT COUNT(*) AS NumberOfAirlinesFromFrance
FROM Airlines
WHERE country = 'France';
```


## 1.3: Adding a few-shot example to the prompt
In this section we will see how we can add a demonstration example to the LLM's prompt.
A demonstartion exaple consists of a question similar to the one we created in the previous task and the reponse that an ideal model would generate.
Such demonstration examples can help the model in two ways:

1. Provide a clear example of the format in which the model is expected to respond. For example if we give an example where the response contains an SQL query and a text exaplanation of how it was created, then the model will most likely also include an explanation in its answer. On the contrary, if the example response only contains the SQL, the model will most likely only generate a SQL query.
2. Provide examples that are similar to the specific question we want the model to answer, to help it generate a more accurate response. Multiple studies have shown that if we dynamically retrieve more similar examples to each given question, the LLM's performance improves.


Let's try to include one static demonstration example to the model's input and observe how it changes its behavior.
Feel free to take an example from the Spider dataset or create one on your own.

In [ ]:
instructions =\
"""
You are an expert in SQL and you always respond with correct SQL queries!
"""

example_question =\
"""
Can you tell me all airport names in the database?
Explain how you arrived at this conclusion.
"""

example_response =\
"""
```sql
SELECT DISTINCT AirportName FROM airport;
```
To get the result, we need to select all the distinct airport names from the airport table.
"""

question =\
"""
Can you tell me all countries that have airport?
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": example_question},
    {"role": "assistant", "content": example_response},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)
print(generated_text)

```sql
SELECT DISTINCT country FROM airport;
```

**Explanation:**
To find all countries that have airports, we select the distinct values in the "country" column from the "airport" table. The `DISTINCT` keyword ensures each country is listed only once, even if multiple airports exist within the same country.


# Part 2: SQL-to-Text

## 2.1: Creating a LLM prompt for SQL-to-Text
Now that we are familiar with Text-to-SQL, it's time to move on to the reverse task of SQL-to-Text.
The goal here is to explain a SQL query in natural language so that the user can understand what it does and be confident that it matches their needs.

In the code block bellow, we will complete the intructions and question parts of the prompt with all the necessary info for the LLM to generate the NL explanation of a SQL.
Similarly to the previous part, we will complete the `instructions` and `question` strings so that they contain all the necessary information for the LLM to generate the desired explanation.


We create a prompt for SQL-to-Text with the example you chose earlier.

In [ ]:
instructions =\
"""
You are an expert in SQL and you always respond with correct SQL queries!
"""

question =\
"""
Can you explain me this SQL query:
"SELECT DISTINCT f.DestAirport FROM flights f
JOIN airport a ON f.SourceAirport = a.AirportCode
WHERE a.Country = 'Greece';" ?
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)
print(generated_text)

Certainly! Here's an explanation of the SQL query:

```sql
SELECT DISTINCT f.DestAirport
FROM flights f
JOIN airport a ON f.SourceAirport = a.AirportCode
WHERE a.Country = 'Greece';
```

**Purpose:**
This query retrieves a list of unique destination airports (`DestAirport`) from the `flights` table where the source airport is located in Greece.

**Detailed breakdown:**

1. **FROM flights f**:
   - Takes data from the `flights` table, aliased as `f` for convenience.

2. **JOIN airport a ON f.SourceAirport = a.AirportCode**:
   - Performs an inner join between the `flights` table and the `airport` table, aliased as `a`.
   - The join condition is that the `SourceAirport` in the `flights` table matches the `AirportCode` in the `airport` table.
   - This links each flight to its originating airport's details.

3. **WHERE a.Country = 'Greece'**:
   - Filters the joined data to include only those source airports that are located in Greece.

4. **SELECT DISTINCT f.DestAirport**:
   - Selects 

## 2.2: Changing the expression of the explanation with Prompt Engineering

Compared to the strictness of SQL, natural language allows us to express the same thing in many different ways.
This means that a query explanation can be phrased in many different ways.
However depending on the user and the use case some expressions might be preferable.
For example, in the context of NLIDBs, we want to provide a brief and compact explanation rather than a long walkthrough of every part of the query.
This is similar to the way NL questions are expressed in the Spider dataset.

One way to achieve this when using a LLM is to provide specific instruction on how we want the LLM to generate the explanation.

We modify the instructions we previously used so that the LLM generates a brief and compact explanation of the SQL query.

In [ ]:
instructions =\
"""
You are an expert in SQL and you always respond with correct SQL queries!
"""

question =\
"""
Can you explain what exactly this SQL query does and how it works:
"SELECT DISTINCT f.DestAirport FROM flights f
JOIN airport a ON f.SourceAirport = a.AirportCode
WHERE a.Country = 'Greece';" ?

Answer in a paragraph
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)
print(generated_text)

This SQL query retrieves a list of unique destination airports (DestAirport) from the flights table, specifically focusing on flights originating from airports located in Greece. It works by joining the 'flights' table (aliased as 'f') with the 'airport' table (aliased as 'a') on the source airport code, ensuring that each flight's source airport matches an entry in the 'airport' table. The WHERE clause filters the results to include only those flights whose source airport is in Greece, as indicated by 'a.Country = 'Greece''. The SELECT DISTINCT ensures that each destination airport appears only once in the output, even if multiple flights are scheduled to the same destination. Essentially, it provides a list of all unique destination airports reachable directly from Greece-based airports.


## 2.3: Changing the expression of the explanation with Few-Shot Demonstrations

As we saw earlier, another way to guide the LLM's behavior towards our goal is to provide similar examples.

We provide few-shot examples that guide the LLM to generate a brief and compact explanation, without explicitely describing the expected behavior in the instructions.

In [ ]:
instructions =\
"""
You are a database assistant.
"""

example_question =\
"""
Can you explain what exactly this SQL query does and how it works:
"SELECT DISTINCT f.DestAirport FROM flights f
JOIN airport a ON f.SourceAirport = a.AirportCode
WHERE a.Country = 'Greece';" ?
"""

example_response =\
"""
**Purpose:**
This query retrieves a list of unique destination airports (`DestAirport`) from the `flights` table where the source airport is located in Greece.

**Detailed breakdown:**

1. **FROM flights f**:
   - Takes data from the `flights` table, aliased as `f` for convenience.

2. **JOIN airport a ON f.SourceAirport = a.AirportCode**:
   - Performs an inner join between the `flights` table and the `airport` table, aliased as `a`.
   - The join condition is that the `SourceAirport` in the `flights` table matches the `AirportCode` in the `airport` table.
   - This links each flight to its originating airport's details.

3. **WHERE a.Country = 'Greece'**:
   - Filters the joined data to include only those source airports that are located in Greece.

4. **SELECT DISTINCT f.DestAirport**:
   - Selects the destination airport codes (`DestAirport`) from the `flights` table.
   - The `DISTINCT` keyword ensures that each destination airport is listed only once, avoiding duplicates.

**Summary:**
The query finds all unique destination airports that can be reached via flights originating from airports in Greece.
"""

question =\
"""
Can you explain what exactly this SQL query does and how it works:
"SELECT DISTINCT DestAirport FROM flights
WHERE SourceAirport IN (
  SELECT AirportCode FROM airports
  WHERE Country = 'Greece');" ?
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": example_question},
    {"role": "assistant", "content": example_response},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)
print(generated_text)

Certainly! Here's an explanation of what this SQL query does and how it works:

---

### Purpose:
This query retrieves a list of **distinct destination airports** (`DestAirport`) from the `flights` table, **specifically for flights that originate from airports located in Greece.**

---

### Breakdown:

1. **Subquery:**
```sql
SELECT AirportCode FROM airports WHERE Country = 'Greece'
```
- **Purpose:** Finds all airport codes (`AirportCode`) of airports situated in Greece.
- It filters the `airports` table to include only those entries where the `Country` is `'Greece'`.
- The result is a list (or set) of airport codes in Greece.

2. **Main query:**
```sql
SELECT DISTINCT DestAirport FROM flights
WHERE SourceAirport IN (
  -- subquery here
);
```
- **Purpose:** Selects all unique destination airports (`DestAirport`) from the `flights` table.
- Uses the `WHERE` clause to filter only those flights where the `SourceAirport` matches **any** airport code from the list generated by the subquer

# Part 3: Data-to-Text

**Data-to-Text** is a general term that describes taking structured data as input and producing natural language that describes them. The two most explored structures in Data-to-Text are tables (Table-to-Text) and graphs (Graph-to-Text).

However, since this notebook focuses on NLIDBs we will explore **Query Results-to-Text** that describes in text the results of an executed query.

In [ ]:
import pandas as pd
import sqlite3

def dataframe_to_markdown(df: pd.DataFrame) -> str:
    """ Utility method to serialize a dataframe to a markdown table """
    return df.to_markdown(index=False)


def execute_query(db_path: str, query: str) -> pd.DataFrame:
    """ Utility method to execute a query on a sqlite database """
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(query, conn)
    return df

## 3.1 Creating the Data-to-Text Prompt

Similar to the previous components we have to define an instruction prompt (system) and the template prompt that we are going to fill with the query results.

So, this time we are going to fill both the system message and the prompt.
We use the given table bellow, containing information about olympic medals of the countries with the most gold medals.
We use the `dataframe_to_markdown` function defined above to turn the dataframe into text, or modify it if you want.

In [ ]:
dataframe = pd.DataFrame(data={"Country": ["USA", "USSR", "China", "Great Britain", "France"], "Gold": [1105, 395, 303, 298, 239], "Silver": [879, 319, 226, 340, 277], "Bronze": [781, 296, 198, 343, 299]})

display(dataframe)

,Country,Gold,Silver,Bronze
0,USA,1105,879,781
1,USSR,395,319,296
2,China,303,226,198
3,Great Britain,298,340,343
4,France,239,277,299


In [ ]:
instructions =\
"""
You are a database analyst. Your goal is describe a table. You always respond with the correct and best answers!
"""

df = dataframe_to_markdown(dataframe)

question =\
f"""
Can you analyse the table {df} and describe it?

Base your answer in the following format:
1. **Purpose:**
2. **Detailed breakdown:**
3. **Summary:**
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)

print(generated_text)

1. **Purpose:**  
The table presents a comparative overview of the total number of Olympic medals—divided into Gold, Silver, and Bronze—awarded to five countries. It aims to highlight the medal achievements of each country across these three categories.

2. **Detailed breakdown:**  
- **USA:** Leading in medal count with 1,105 Gold medals, 879 Silver medals, and 781 Bronze medals, indicating a strong overall performance across all medal types.  
- **USSR:** Second highest in total medals, with 395 Gold, 319 Silver, and 296 Bronze, showing significant competitive success, especially in Gold medals relative to other countries outside the USA.  
- **China:** With 303 Gold, 226 Silver, and 198 Bronze medals, China has a considerable medal tally, particularly excelling in Gold medals relative to other countries like Great Britain and France.  
- **Great Britain:** Has 298 Gold, 340 Silver, and 343 Bronze medals, notably possessing the highest Silver and Bronze medals among the countries lis

## 3.2 Query Results from an executed query

In a scenario closer to real world, we would not have hardcoded examples but instead we would execute a query (written by a user or generated from a Text-to-SQL model).

In the following cell, we combine the utility methods `execute_query` and `data_to_text` and manually write a query for the `sqlite` database found in `/content/spider_data/database/cinema/cinema.sqlite`.

We can check how the schema looks like in the `schema_dict` or in:
`/content/spider_data/database/cinema/schema.sql`

The steps we should follow are:
1. Write a SQL query and test what results it returns using the `execute_query` method
2. Pass the results into the `dataframe_to_markdown` method and automatically create the "question" part of the prompt
3. Observe the results, experiment with the prompt template above

We can improve the prompt by providing extra information that would help the model into the prompt. Some ideas:
* The SQL query
* The names of the tables in the queries

We observe and document, then, if the verbalisation improves or gets worse.

In [ ]:
# We fill the code following the steps above
instructions =\
"""
You are a database analyst. Your goal is describe a table. You always respond with the correct and best answers!
"""

db_path = "/content/spider_data/database/cinema/cinema.sqlite"
query = "SELECT Name FROM cinema WHERE Openning_year > 2010;"

results = execute_query(db_path, query)
df = dataframe_to_markdown(results)

question =\
f"""
Can you analyse the table {df} and describe it?

Base your answer in the following format:
1. **Purpose:**
2. **Detailed breakdown:**
3. **Summary:**
"""

conversation = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question},
]

generated_text = llm_generate(conversation)

print(generated_text)

1. **Purpose:**
The table appears to list names of locations, sites, or possibly geographical or organizational entities, most likely within a specific country or region.

2. **Detailed breakdown:**
- The table contains a single column labeled "Name."
- The entries include:
  - **Carrowleagh:** Likely a place, possibly a townland, village, or estate in Ireland.
  - **Dublin Array:** Possibly a reference to a site or a specific geographical feature associated with Dublin, or a named area.
  - **Grouse Lodge:** Known as a resort, retreat, or event venue, potentially in Ireland.
  - **Moneypoint:** A well-known location in Ireland, specifically a major power station and port.
  - **Mount Callan:** Likely an Irish geographic feature, such as a mountain or hill.
  - **Oriel:** Could refer to the Oriel region, a historical and geographical area in Ireland.

3. **Summary:**
The table lists various notable locations or entities, predominantly associated with Ireland. The entries include geogra

# Part 4: Combining everything together

In this final section, we will orchestrate all 3 components mentioned above:

* Text-to-SQL
* SQL-to-Text
* Query Results-to-Text

into a single method.

Our method will take as input:

1. a natural language question
2. a db path from the Spider dataset

Then it will:
1. Translate the natural language into SQL
2. Explain the generated SQL using SQL-to-Text
3. Execute the query and call the Query Results-to-Text model

We can document your findings concerning:
* Performance for each component but combined also
* Latency of the system
* Possible suggestions of ways to improve each system

In [ ]:
# We fill the code following similar steps as above

def combine_everything(question: str, db_path: str) -> str:
  """ Utility method to execute Text-to-SQL, SQL-to-Text and Query Results-to-Text """

  # 1. Translate natural language into SQL
  nl_to_sql_instructions =\
  """
  You are an expert in SQL and you always respond with correct SQL queries!
  Only respond with the SQL query and nothing else.
  """

  nl_to_sql_question =\
  f"""
  Generate a SQL query for the following natural language question: {question}
  The schema is this: {'/content/spider_data/database/cinema/schema.sql'}
  """

  conversation = [
      {"role": "system", "content": nl_to_sql_instructions},
      {"role": "user", "content": nl_to_sql_question},
  ]

  sql_generated_query = llm_generate(conversation)
  print("Generated SQL:")
  print(sql_generated_query)
  print()

  # 2. Explain the generated SQL using SQL-to-Text
  sql_to_text_instructions =\
  """
  You are a database assistant.
  Explain the following SQL query in a brief and compact paragraph.
  """

  sql_to_text_question =\
  f"""
  Can you explain what exactly this SQL query does:
  {sql_generated_query}?

  Answer in a paragraph
  """

  conversation = [
      {"role": "system", "content": sql_to_text_instructions},
      {"role": "user", "content": sql_to_text_question},
  ]

  generated_sql_to_text = llm_generate(conversation)
  print("SQL explanation:")
  print(generated_sql_to_text)
  print()

  # 3. Execute the query and call the Query Results-to-Text model
  db_to_text_instructions =\
  """
  You are a database analyst. Your goal is describe a table. You always respond with the correct and best answers!
  Describe the provided table data based on its contents.
  """

  try:
    results = execute_query(db_path, sql_generated_query)
    df = dataframe_to_markdown(results)

    db_to_text_question =\
    f"""
    Can you analyse the following table {df} and describe it?

    Base your answer in the following format:
    1. **Purpose:**
    2. **Detailed breakdown:**
    3. **Summary:**
    """

    conversation = [
        {"role": "system", "content": db_to_text_instructions},
        {"role": "user", "content": db_to_text_question},
    ]

    generated_db_to_text = llm_generate(conversation)
    print("Query results explanation:")
    print(generated_db_to_text)
    print()

  except Exception as e:
      print(f"Error executing query or generating data description: {e}")
      print(f"Attempted SQL query: {sql_generated_query}")

In [ ]:
# Putting it all together
db_path = "/content/spider_data/database/cinema/cinema.sqlite"
query_2 = "SELECT Name FROM cinema WHERE Openning_year > 2010 AND Capacity > 300;"

combine_everything(query_2, db_path)

Generated SQL:
SELECT Name FROM cinema WHERE Openning_year > 2010 AND Capacity > 300;

SQL explanation:
This SQL query retrieves the names of cinemas from the "cinema" table that opened after the year 2010 and have a capacity of more than 300 seats. It filters the data based on two conditions: the "Openning_year" must be greater than 2010, and the "Capacity" must be greater than 300, and then it returns the "Name" of those cinemas that meet both criteria.

Query results explanation:
1. **Purpose:**  
   The table appears to list a set of geographic or locality names, possibly representing places such as towns, neighborhoods, or regions.

2. **Detailed breakdown:**  
   The table includes four entries: "Carrowleagh," "Dublin Array," "Mount Callan," and "Oriel." The names suggest they are place names, with some referencing well-known locations (e.g., Dublin refers to the capital city of Ireland). "Dublin Array" may refer to a geographical feature or a specific area within or near Dublin.